FIR　EEG-ECoG 共通 EEG 予測 共通

1. FIR 0, IIR 1, LSTM 2
2. EEG-ECoG間でフィルター共通 0, バラバラ 1
3. 予測対象 EEG 0, ECoG 1
4. 予測の仕方 共通 0, NNをわける（フォルター特性が変わる） 1



In [64]:
# -*- coding: utf_8 -*-  
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn

import copy
import json
import pandas as pd
import numpy as np
import datetime
import math
import time

import chainer
from chainer import cuda, Function, gradient_check, utils, Variable
from chainer import optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from scipy import signal


import pickle
import sys
import random
import h5py

import nn_class

In [65]:
test=h5py.File('Ma/DeepANES.mat')

ECoG = np.array(test['ECoG'])
EEG = np.array(test['EEG'])
EEG = np.delete(EEG, 14, axis=1)
EEG = EEG.astype('float32')
ECoG = ECoG.astype('float32')

In [66]:
order = 5
batch_size = 1000
time_range = 1000*10
start = 0

train_time = [start, time_range+start]
hamming = signal.hamming(train_time[1]-train_time[0])
#training ぶんだけにする
#EEG_t = np.zeros((EEG.shape[0]-order+1, EEG.shape[1], order))
#ECoG_t = np.zeros((ECoG.shape[0]-order+1, ECoG.shape[1], order))
EEG_t = np.zeros((train_time[1]-train_time[0], EEG.shape[1], order))
ECoG_t = np.zeros((train_time[1]-train_time[0], ECoG.shape[1], order))
EEG_tmp = EEG[train_time[0]:train_time[1], :] *np.array([hamming]*EEG.shape[1]).T
ECoG_tmp = ECoG[train_time[0]:train_time[1], :] *np.array([hamming]*ECoG.shape[1]).T
for t in range(order):
    for ch in range(EEG.shape[1]):
        #EEG_t[:,ch,t] = EEG[:EEG.shape[0]-order+1, ch] 
        EEG_t[t:,ch,t] = EEG_tmp[:time_range-t, ch] 
    for ch in range(ECoG.shape[1]):
        #ECoG_t[:,ch,t] = ECoG[:ECoG.shape[0]-order+1, ch] 
        ECoG_t[t:,ch,t] = ECoG_tmp[:time_range-t, ch] 
EEG_t = Variable((EEG_t).astype('float32'))
ECoG_t = Variable((ECoG_t).astype('float32'))

In [67]:
def calNN(timeSpacePerceptron, EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, loop=200):
    loss_history = np.zeros((loop, 1))
    optimizer = optimizers.AdaDelta()
    #optimizer = optimizers.AdaDelta()
    optimizer.setup(timeSpacePerceptron)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.05))
    #optimizer.add_hook(chainer.optimizer.Lasso(0.5))
    #optimizer.add_hook(chainer.optimizer.GradientClipping(0.5))
    timeSpacePerceptron.zerograds()
    for i in range(loop):
        timeSpacePerceptron.zerograds()
        x1, x2, EEG_filtered_prev, ECoG_filtered_prev = timeSpacePerceptron(EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, train=True)
        #loss = F.cross_covariance(x1, x2)
        loss = F.mean_squared_error(x1, x2)
        loss.backward()
        optimizer.prepare()
        optimizer.update()
        loss_history[i] = loss.data
    return timeSpacePerceptron, loss_history, EEG_filtered_prev, ECoG_filtered_prev, x1, x2

In [96]:
def loop_calNN(EEG_t, ECoG_t, data_type, loop=200, loop_num=20):
    EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
    ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ
    saved_data = dict()
    #outout = []
    out_loss_history = []
    for n in range(loop_num):
        out = []
        loss_history = np.zeros((loop, 1))
        timeSpacePerceptron = nn_class.__dict__["TimeSpacePerceptron{0}".format(data_type)](time_range)
        timeSpacePerceptron, loss_history, EEG_filtered_prev, ECoG_filtered_prev, x1, x2 = calNN(timeSpacePerceptron, EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, loop=loop)
        out_loss_history.append(loss_history)
        #outout.append(timeSpacePerceptron)
    saved_data['loss_history'] = out_loss_history
    saved_data['NN'] = timeSpacePerceptron
    #saved_data['NN'] = outout
    return saved_data, EEG_filtered_prev, ECoG_filtered_prev

def loop_calNN_batch(EEG_t, ECoG_t, data_type, loop_num=20, batch_size=1000):
    saved_data = dict()
    #outout = []
    out_loss_history = []
    for n in range(loop_num):
        for b in range(int(EEG_t.data.shape[0]/batch_size)):
            t1 = b*batch_size
            t2 = (b+1)*batch_size-1
            EEG_filtered_prev = [Variable(np.zeros((batch_size-1, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
            ECoG_filtered_prev = [Variable(np.zeros((batch_size-1, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ
            out = []
            timeSpacePerceptron = nn_class.__dict__["TimeSpacePerceptron{0}".format(data_type)](batch_size-1)
            timeSpacePerceptron, loss_history, EEG_filtered_prev, ECoG_filtered_prev, x1, x2 = calNN(timeSpacePerceptron, EEG_t[t1:t2,:,:], ECoG_t[t1:t2,:,:], EEG_filtered_prev, ECoG_filtered_prev, loop=loop)
            out_loss_history.append(loss_history)
            #outout.append(timeSpacePerceptron)
    saved_data['loss_history'] = out_loss_history
    saved_data['NN'] = timeSpacePerceptron
    #saved_data['NN'] = outout
    return saved_data, EEG_filtered_prev, ECoG_filtered_prev

mini batch で 1sec のデータを1min まわせるようにする

In [ ]:
saved_data = dict()
#loop = 100
loop_num = 100
st = time.time()


# channle をわけない

data_type ='0000'
saved_data[data_type],  EEG_filtered_prev, ECoG_filtered_prev =  loop_calNN_batch(EEG_t, ECoG_t, data_type, loop_num=loop_num, batch_size=batch_size)

data_type ='1000'
saved_data[data_type],  EEG_filtered_prev, ECoG_filtered_prev =  loop_calNN_batch(EEG_t, ECoG_t, data_type, loop_num=loop_num, batch_size=batch_size)

#data_type ='2000'
#saved_data[data_type],  EEG_filtered_prev, ECoG_filtered_prev =  loop_calNN_batch(EEG_t, ECoG_t, data_type, loop_num=loop_num, batch_size=batch_size)


print()
print(time.time()-st)

In [ ]:
data_type ='1000'
corr_val = np.zeros((16, int(EEG_t.data.shape[0]/batch_size)))
for b in range(int(EEG_t.data.shape[0]/batch_size)):
    EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
    ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ
    t1 = b*batch_size
    t2 = (b+1)*batch_size-1
    x1, x2, EEG_filtered_prev, ECoG_filtered_prev = saved_data[data_type]['NN'](EEG_t[t1:t2,:,:], ECoG_t[t1:t2,:,:], EEG_filtered_prev, ECoG_filtered_prev, train=False)
    for ch in range(16):
        corr_val[ch, b] = np.corrcoef(x1.data[:,ch], x2.data[:,ch])[0, 1]


In [ ]:
plt.plot(corr_val)

In [60]:
saved_data = dict()
loop = 100
loop_num = 1
st = time.time()

# channle をわけない

data_type ='0000'
saved_data[data_type] =  loop_calNN(EEG_t, ECoG_t, data_type, loop=loop, loop_num=loop_num)

data_type ='1000'
saved_data[data_type],  EEG_filtered_prev, ECoG_filtered_prev =  loop_calNN(EEG_t, ECoG_t, data_type, loop=loop, loop_num=loop_num)

data_type ='2000'
saved_data[data_type],  EEG_filtered_prev, ECoG_filtered_prev =  loop_calNN(EEG_t[:,:,0], ECoG_t[:,:,0], data_type, loop=loop, loop_num=loop_num)

print()
print(time.time()-st)


158.10014009475708


/Users/vv000844/.pyenvi/versions/anaconda3-2.5.0/lib/python3.5/site-packages/chainer/functions/activation/lstm.py:15: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + numpy.exp(-x))


In [82]:
data_type ='1000'

EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ

x1, x2, EEG_filtered_prev, ECoG_filtered_prev = saved_data[data_type]['NN'](EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, train=False)


for ch in range(16):
    print(np.corrcoef(x1.data[:,ch], x2.data[:,ch])[0, 1])

InvalidType: 
Invalid operation is performed in: Concat (Forward)

Expect: in_types[0].shape[0] == in_types[1].shape[0]
Actual: 10000 != 999

In [63]:
data_type ='2000'

EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ

x1, x2, EEG_filtered_prev, ECoG_filtered_prev = saved_data[data_type]['NN'](EEG_t[:,:,0], ECoG_t[:,:,0], EEG_filtered_prev, ECoG_filtered_prev, train=False)


for ch in range(16):
    print(np.corrcoef(x1.data[:,ch], x2.data[:,ch])[0, 1])

-0.0416869389259
0.130948456042
0.0700970742688
0.311045227319
0.0378985544823
0.0535104646283
-0.0233822023459
0.0578037100954
-0.0258820790615
nan
0.0715365301592
0.0878812351078
0.41289599889
-0.000382946270612
0.357790984888
0.105311732183


/Users/vv000844/.pyenvi/versions/anaconda3-2.5.0/lib/python3.5/site-packages/chainer/functions/activation/lstm.py:15: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + numpy.exp(-x))
/Users/vv000844/.pyenvi/versions/anaconda3-2.5.0/lib/python3.5/site-packages/numpy/lib/function_base.py:2558: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/vv000844/.pyenvi/versions/anaconda3-2.5.0/lib/python3.5/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


1. 各EEG の相関係数　一番マシなやつはどれか 1 秒ごとの相関係数を計測すべき
あとはデータセットごとの成績（一番マシなモデルで）
一番マシなモデルでのECoG予測
    有意に成績が高いところの空間周波数分布を可視化
2. EEG の方にも空間フィルターをはさみ、CCA っぽくしたやつで EV←いるか？

In [47]:
start = 12000

train_time = [start, time_range+start]
hamming = signal.hamming(train_time[1]-train_time[0])
#training ぶんだけにする
EEG_t = np.zeros((train_time[1]-train_time[0], EEG.shape[1], order))
ECoG_t = np.zeros((train_time[1]-train_time[0], ECoG.shape[1], order))
EEG_tmp = EEG[train_time[0]:train_time[1], :] *np.array([hamming]*EEG.shape[1]).T
ECoG_tmp = ECoG[train_time[0]:train_time[1], :] *np.array([hamming]*ECoG.shape[1]).T
for t in range(order):
    for ch in range(EEG.shape[1]):
        EEG_t[t:,ch,t] = EEG_tmp[:time_range-t, ch] 
    for ch in range(ECoG.shape[1]):
        ECoG_t[t:,ch,t] = ECoG_tmp[:time_range-t, ch] 
EEG_t = Variable((EEG_t).astype('float32'))
ECoG_t = Variable((ECoG_t).astype('float32'))

In [48]:
data_type ='0000'

EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ

x1, x2, EEG_filtered_prev, ECoG_filtered_prev = saved_data[data_type][0]['NN'](EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, train=False)


for ch in range(16):
    print(np.corrcoef(x1.data[:,ch], x2.data[:,ch])[0, 1])


0.0207187471635
0.0458717261303
-0.0108192695865
0.0661329574022
-0.034606745611
0.0227852900663
-0.0326817656876
0.0546924543054
-0.0116927409141
0.0318454567344
0.0721525861468
0.0412125716064
0.00348209575144
0.0135745480717
0.0102102233428
0.018274019195


In [49]:
data_type ='1000'

EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ

x1, x2, EEG_filtered_prev, ECoG_filtered_prev = saved_data[data_type]['NN'](EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, train=False)


for ch in range(16):
    print(np.corrcoef(x1.data[:,ch], x2.data[:,ch])[0, 1])

-0.0449766211516
-0.0232906095272
-0.0772777301383
-0.0318539540555
-0.0241463867009
-0.0160971199035
-0.0437096095499
-0.0160508815258
0.0394506079178
-0.0147266687392
-0.121383296745
-0.0159970997966
-0.0150179434717
-0.0382470762072
-0.0824070276322
-0.0562502967309


In [50]:
data_type ='2000'

EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ

x1, x2, EEG_filtered_prev, ECoG_filtered_prev = saved_data[data_type]['NN'](EEG_t[:,:,0], ECoG_t[:,:,0], EEG_filtered_prev, ECoG_filtered_prev, train=False)


for ch in range(16):
    print(np.corrcoef(x1.data[:,ch], x2.data[:,ch])[0, 1])

-0.119102555124
-0.278238648829
-0.116820860403
-0.080254958995
-0.29244663412
-0.0435385437061
0.047716906933
0.112464034367
-0.106278167708
-0.0720733619743
0.316266828744
-0.0312669475964
0.152641354079
-0.0879843893547
-0.336172864583
-0.0311284166425


/Users/vv000844/.pyenvi/versions/anaconda3-2.5.0/lib/python3.5/site-packages/chainer/functions/activation/lstm.py:15: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + numpy.exp(-x))
